# Estimagic with SciPy's `least_squares`

This is the accompanying documentation to the Pull Request adding the scipy [least_squares](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html) to estimagic.

## Steps taken
To include the optimizer, I conducted the following steps:

**TODO: Co through commits, check what was done, descibe here**
1. Blub
2. Blub
3. Fix the broken link `https://gitlab.com/petsc/petsc/-/tree/master/src/binding/petsc4py/`, so the documentation builds


## Sphere Example
To show that the optimizer works within estimagic, I've included the [Sphere Example](https://estimagic.readthedocs.io/en/latest/getting_started/first_optimization_with_estimagic.html) from the documentation.


In [1]:
import numpy as np
import pandas as pd
from estimagic import minimize
from estimagic.logging.read_log import read_optimization_iteration

/home/yana/PycharmProjects/estimagic/venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def sphere(params):
    """Spherical criterion function.

    The unique local and global optimum of this function is at
    the zero vector. It is differentiable, convex and extremely
    well behaved in any possible sense.

    Args:
        params (pandas.DataFrame): DataFrame with the columns
            "value", "lower_bound", "upper_bound" and potentially more.

    Returns:
        dict: A dictionary with the entries "value" and "root_contributions".

    """
    out = {
        "value": (params["value"] ** 2).sum(),
        "root_contributions": params["value"],
    }
    return out


def sphere_gradient(params):
    """Gradient of spherical criterion function"""
    return params["value"] * 2

In [3]:
start_params = pd.DataFrame(
    data=np.arange(5) + 1,
    columns=["value"],
    index=[f"x_{i}" for i in range(5)],
)
params_with_bounds = start_params.copy()

params_with_bounds["lower_bound"] = [0, 1, 0, -1, 0]
params_with_bounds["upper_bound"] = [np.inf] * 5


In [4]:
# With bounds
res = minimize(
    criterion=sphere,
    params=params_with_bounds,
    algorithm="scipy_least_squares",
    derivative=sphere_gradient
)
res["solution_params"].round(2)

,lower_bound,upper_bound,value
x_0,0.0,inf,0.0
x_1,1.0,inf,1.0
x_2,0.0,inf,0.0
x_3,-1.0,inf,-0.0
x_4,0.0,inf,0.0


In [5]:
# With bounds and constraints
constraints = [{"loc": ["x_0", "x_3"], "type": "fixed", "value": [1, 4]}]
res = minimize(
    criterion=sphere,
    params=params_with_bounds,
    algorithm="scipy_least_squares",
    derivative=sphere_gradient,
    constraints=constraints,
)
res["solution_params"].round(2)

,lower_bound,upper_bound,value
x_0,0.0,inf,1.0
x_1,1.0,inf,1.0
x_2,0.0,inf,0.0
x_3,-1.0,inf,4.0
x_4,0.0,inf,0.0


## Supplying Arguments
The included `least_squares` optimizer takes arguments (see documentation).
This example sets the internal algorithm to `dogbox`.

In [6]:
# Use the dogbox algorithm instead of the default
algo_options = {
    'method': 'dogbox'
}

res = minimize(
    criterion=sphere,
    params=start_params,
    algorithm="scipy_least_squares",
    derivative=sphere_gradient,
    algo_options=algo_options
)
res["solution_params"].round(2)

,lower_bound,upper_bound,value
x_0,-inf,inf,0.0
x_1,-inf,inf,0.0
x_2,-inf,inf,0.0
x_3,-inf,inf,0.0
x_4,-inf,inf,0.0


## Rosenbrock Example
To further illustrate the optimizers capabilities, I've also included the Rosenbrock example from the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html).

### The Rosenbrock Function
In mathematical optimization, the Rosenbrock function is a non-convex function, introduced by Howard H. Rosenbrock in 1960, which is used as a performance test problem for optimization algorithms. It is also known as Rosenbrock's valley or Rosenbrock's banana function. [[Rosenbrock]](https://academic.oup.com/comjnl/article/3/3/175/345501)

In [7]:
def fun_rosenbrock(params):
    x = params["value"]
    res = 10 * (x[1] - x[0]**2)**2 + (1 - x[0])**2
    return {
        "value": res,
        "root_contributions": x
    }
def jac_rosenbrock(params):
    x, y = params["value"]
    return [
        2 * (20 * x**3 - 20 * x * y + x - 1), # diff. by x
        20 * (y - x**2)  # diff. by y
    ]

In [8]:
x0_rosenbrock = pd.DataFrame(
    data=[2, 2],
    columns=["value"],
    index=[f"x_{i}" for i in range(2)],
)

In [9]:
# Without bounds
res = minimize(
    criterion=fun_rosenbrock,
    params=x0_rosenbrock,
    algorithm="scipy_least_squares",
    derivative=jac_rosenbrock,
)
# Function has minima at (1, 1)
res["solution_params"].round(2)

,lower_bound,upper_bound,value
x_0,-inf,inf,1.0
x_1,-inf,inf,1.0


In [ ]:
# With bounds
x0_rosenbrock["lower_bound"] = [-np.inf, 1.5]
x0_rosenbrock["upper_bound"] = [np.inf] * 2
res = minimize(
    criterion=fun_rosenbrock,
    params=x0_rosenbrock,
    algorithm="scipy_least_squares",
    derivative=jac_rosenbrock,
)
# Function has new solution that lies on the bound (1.22, 1.5)
res["solution_params"].round(2)